In [1]:
import pandas as pd
import numpy as np
import os
import time
import glob
import pickle

In [4]:
count = 1
# data = []
mini = dict()
maxi = dict()
start = time.time()
stock_names = []
for file in glob.glob('Data/Stocks/*.us.txt'):
    if(os.stat(file).st_size != 0): # if the file is NOT empty
        tempdf = pd.read_csv(file)
        stock_name = str.upper(file[12:-7:1]) # Stock Name
        stock_names.append(stock_name)
        tempdf['Stock'] = stock_name
        tempdf = tempdf[['Date','Open','High','Low','Close','Volume','Stock']] # we keep the columns that we want 

#       Local  Min - Max====================================================================
        m1 =pd.DataFrame()
        m2 = pd.DataFrame()
        m1['Price'] = tempdf.Low[(tempdf.Low.shift(1) > tempdf.Low) & (tempdf.Low.shift(-1) > tempdf.Low)]
        m1['Date'] = tempdf.Date[(tempdf.Low.shift(1) > tempdf.Low) & (tempdf.Low.shift(-1) > tempdf.Low)]
        m1['Volume'] = tempdf.Volume[(tempdf.Low.shift(1) > tempdf.Low) & (tempdf.Low.shift(-1) > tempdf.Low)]
        m2['Price'] = tempdf.High[(tempdf.High.shift(1) < tempdf.High) & (tempdf.High.shift(-1) < tempdf.High)]
        m2['Date'] = tempdf.Date[(tempdf.High.shift(1) < tempdf.High) & (tempdf.High.shift(-1) < tempdf.High)]
        # Valid Values
        m1 = m1[m1.Price >0]
        m2 = m2[m2.Price >0]
        
        mini[stock_name] = m1.reset_index(drop=True)
        maxi[stock_name] = m2.reset_index(drop=True)
    count += 1
    if count%200 == 0:
        print('Progress:',count,'/ 7195 = ',int(100*count/7195),'%  Time = ', int(time.time() - start)//60,'min')  
print(count)
# data = pd.concat(data)
# data.reset_index(inplace=True, drop=True)


Progress: 200 / 7195 =  2 %  Time =  0 min
Progress: 400 / 7195 =  5 %  Time =  0 min
Progress: 600 / 7195 =  8 %  Time =  0 min
Progress: 800 / 7195 =  11 %  Time =  0 min
Progress: 1000 / 7195 =  13 %  Time =  0 min
Progress: 1200 / 7195 =  16 %  Time =  0 min
Progress: 1400 / 7195 =  19 %  Time =  0 min
Progress: 1600 / 7195 =  22 %  Time =  0 min
Progress: 1800 / 7195 =  25 %  Time =  0 min
Progress: 2000 / 7195 =  27 %  Time =  0 min
Progress: 2200 / 7195 =  30 %  Time =  0 min
Progress: 2400 / 7195 =  33 %  Time =  1 min
Progress: 2600 / 7195 =  36 %  Time =  1 min
Progress: 2800 / 7195 =  38 %  Time =  1 min
Progress: 3000 / 7195 =  41 %  Time =  1 min
Progress: 3200 / 7195 =  44 %  Time =  1 min
Progress: 3400 / 7195 =  47 %  Time =  1 min
Progress: 3600 / 7195 =  50 %  Time =  1 min
Progress: 3800 / 7195 =  52 %  Time =  1 min
Progress: 4000 / 7195 =  55 %  Time =  1 min
Progress: 4200 / 7195 =  58 %  Time =  1 min
Progress: 4400 / 7195 =  61 %  Time =  1 min
Progress: 4600 / 

In [9]:
tot_names = len(stock_names)

In [10]:
Lows, Highs = dict(), dict()
start = time.time()
timeLimit = 10
count = 0
# testlist = ['ZYNE']
for stockName in stock_names:

    indmin = []#set()
    indmax = []#set()
    ind=0
    small, big = mini[stockName] , maxi[stockName]
    si, bi = 0 , 0 # small index, big index
    SurvMin, SurvMax = [],[]#set(), set() # index list of survivors
    smallLen, bigLen = len(small), len(big)
    
    stopMin, stopMax, tooSmall =False, False, False
    
    if smallLen < 4 or bigLen<4: 
        tooSmall, stopMin, stopMax = True, True, True
    
    if tooSmall == False:
        # we find the first local max after the first local min
        while big.iloc[bi]['Date'] < small.iloc[0]['Date'] and bi < bigLen-2:    
            bi += 1

    while stopMin == False and stopMax == False:
        # Minimun
        minmin = small.iloc[si]['Price']
        minminInd = si
        addmin = False
        while small.iloc[si]['Date'] < big.iloc[bi]['Date'] and stopMin == False:
            # Find the min of the consecutive minima
            if small.iloc[si]['Price'] < minmin:
                minmin = small.iloc[si]['Price']
                minminInd = si
            if si +1 >= smallLen: 
                stopMin = True
            else:
                si += 1
                addmin =True


        indmin.append(ind)#.add(ind)  
        ind += 1
        SurvMin.append(minminInd)#add(minminInd)
        if addmin == False: si += 1

        # Maximun
        maxmax = big.iloc[bi]['Price']
        maxmaxInd = bi
        addmin = False
        if stopMin == True:
            while bi < bigLen -1:
                if big.iloc[bi]['Price'] > maxmax:
                    maxmax = big.iloc[bi]['Price']
                    maxmaxInd = bi
                bi += 1
                addmax = True

        else:        
            while big.iloc[bi]['Date'] <= small.iloc[si]['Date'] and stopMax == False: 
                # Find the max of the consecutive maxima
                if big.iloc[bi]['Price'] > maxmax:
                    maxmax = big.iloc[bi]['Price']
                    maxmaxInd = bi
                if bi +1 >= bigLen: 
                    stopMax = True
                else:
                    bi += 1
                    addmax =True


        indmax.append(ind)#add(ind)
        ind += 1
        SurvMax.append(maxmaxInd)#add(maxmaxInd)
        if addmax == False: bi += 1


    if tooSmall == False:
        Lows[stockName] = small[small.index.isin(SurvMin)].set_index(pd.Index(indmin))
        Highs[stockName] = big[big.index.isin(SurvMax)].set_index(pd.Index(indmax))
    else:
        Lows[stockName] = pd.DataFrame()
        Highs[stockName] = pd.DataFrame()
        
    count += 1
    if count%100 == 0: # 105 min
        print('Progress:',count,'/ ',str(tot_names),' = ',int(100*count/tot_names),'%  Time = ', int(time.time() - start)//60,'min')  
print(count) 



Progress: 100 /  7163  =  1 %  Time =  1 min
Progress: 200 /  7163  =  2 %  Time =  3 min
Progress: 300 /  7163  =  4 %  Time =  4 min
Progress: 400 /  7163  =  5 %  Time =  6 min
Progress: 500 /  7163  =  6 %  Time =  7 min
Progress: 600 /  7163  =  8 %  Time =  9 min
Progress: 700 /  7163  =  9 %  Time =  10 min
Progress: 800 /  7163  =  11 %  Time =  12 min
Progress: 900 /  7163  =  12 %  Time =  13 min
Progress: 1000 /  7163  =  13 %  Time =  15 min
Progress: 1100 /  7163  =  15 %  Time =  16 min
Progress: 1200 /  7163  =  16 %  Time =  18 min
Progress: 1300 /  7163  =  18 %  Time =  19 min
Progress: 1400 /  7163  =  19 %  Time =  21 min
Progress: 1500 /  7163  =  20 %  Time =  22 min
Progress: 1600 /  7163  =  22 %  Time =  24 min
Progress: 1700 /  7163  =  23 %  Time =  26 min
Progress: 1800 /  7163  =  25 %  Time =  27 min
Progress: 1900 /  7163  =  26 %  Time =  28 min
Progress: 2000 /  7163  =  27 %  Time =  29 min
Progress: 2100 /  7163  =  29 %  Time =  31 min
Progress: 2200

In [17]:
def pickleSave(obj,name):
    with open(name+'.pkl', 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def pickleLoad(name):
    with open(name+'.pkl', 'rb') as input:
        return pickle.load(input)
    

In [18]:
start = time.time()
pickleSave(Lows, 'LowsValid')
print('Time:', time.time() - start )

Time: 3.1751818656921387


In [19]:
start = time.time()
pickleSave(Highs, 'HighsValid')
print('Time:', time.time() - start )

Time: 2.284130811691284
